In [1]:
import glob, os
import pandas as pd

file_list = glob.glob('./data/*.csv') 
df = pd.DataFrame()

for f in file_list:
    df_tmp = pd.read_csv(f)
    i = f.find('cy')
    df_tmp['year'] = f[i+2:i+6]
    i = f.find('\\')
    df_tmp['sex'] = f[i+1]
    df = pd.concat([df, df_tmp],ignore_index=True)

In [9]:
df.head()

,Given Name,Amount,year,sex
0,MARGARET,341,1944,f
1,HELEN,209,1944,f
2,JUDITH,189,1944,f
3,PATRICIA,186,1944,f
4,PAMELA,160,1944,f


In [4]:
df.drop('Position', axis=1, inplace=True)

In [5]:
df.head()

,Given Name,Amount,year,sex
0,MARGARET,341,1944,f
1,HELEN,209,1944,f
2,JUDITH,189,1944,f
3,PATRICIA,186,1944,f
4,PAMELA,160,1944,f


In [6]:
dups = df[df.duplicated(['Given Name','year'],keep=False)].groupby(['Given Name','year']).Amount.sum()

In [15]:
df_dup = dups.to_frame()

In [16]:
df_dup.head()

Amount
Given Name year        
(NOT       1968       8
AARON      1963       8
           1966       7
           1968      21
           1969      10

In [8]:
df.shape[0]

210939

In [12]:
df.duplicated(['Given Name','year'],keep=False).value_counts()

False    193224
True      17715
dtype: int64

In [13]:
df.drop_duplicates(['Given Name','year'],inplace=True)

In [14]:
df.duplicated(['Given Name','year'],keep=False).value_counts()

False    201935
dtype: int64

In [17]:
df2 = pd.merge(df, df_dup, on=['Given Name', 'year'])

In [26]:
df_dup.reset_index(level=['Given Name', 'year'], inplace=True)

In [28]:
df_dup[(df_dup['Given Name'] == 'SAMUEL') & (df_dup['year'] == '1979')]

,Given Name,year,Amount
7204,SAMUEL,1979,94


In [30]:
df2.shape[0]

8711

In [13]:
df.to_csv("all_names.csv")

you need to merge rows if name is the same!!!